In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet
from keras.layers import Input, Subtract, Dense, Lambda, Concatenate
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import keras.backend as K
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('train.csv')

all_right_images = df['right'].tolist()

new_data = []

for index, row in df.iterrows():
    left_image = row['left']
    true_match = row['right']
    
    random_images = [img for img in all_right_images if img != true_match]
    random_images = random.sample(random_images, 19)
    
    position = random.randint(0, 19)
    random_images.insert(position, true_match)
    
    new_entry = {'left': left_image, 
                 'c0': random_images[0], 
                 'c1': random_images[1], 
                 'c2': random_images[2], 
                 'c3': random_images[3], 
                 'c4': random_images[4], 
                 'c5': random_images[5], 
                 'c6': random_images[6], 
                 'c7': random_images[7], 
                 'c8': random_images[8], 
                 'c9': random_images[9],
                 'c10': random_images[10], 
                 'c11': random_images[11], 
                 'c12': random_images[12], 
                 'c13': random_images[13], 
                 'c14': random_images[14],
                 'c15': random_images[15], 
                 'c16': random_images[16], 
                 'c17': random_images[17], 
                 'c18': random_images[18], 
                 'c19': random_images[19],
                 'groundtruth': true_match}
    
    new_data.append(pd.DataFrame([new_entry]))

new_df = pd.concat(new_data, ignore_index=True)

new_df.to_csv('newTrain.csv', index=False)

In [3]:
base_model = resnet.ResNet50(
    weights="imagenet", include_top=False, pooling='avg')
)

base_model.trainable = False

def build_model(base_model):
    left_input = Input(shape=(224, 224, 3))
    right_input = Input(shape=(224, 224, 3))

    left_embedding = base_model(left_input)
    right_embedding = base_model(right_input)
    
    left_flatten = layers.Flatten()(left_embedding)
    right_flatten = layers.Flatten()(right_embedding)

    merged = Concatenate()([left_flatten, right_flatten])

    output = Dense(20, activation='softmax')(merged)

    model = Model(inputs=[left_input, right_input], outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def preprocess_image(img_path, is_left=True):
    img_path = img_path + ".jpg"
    if len(img_path) == 7:
        if is_left:
            img_path = "./train/left/" + img_path
        else:
            img_path = "./train/right/" + img_path 
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(expanded_img_array)

def extract_embedding(img_name, folder):
    img = preprocess_image(img_name, folder)
    return base_model.predict(img)

In [4]:
def data_generator(df, batch_size=32):
    while True:
        batch_indices = np.random.choice(a=len(df), size=batch_size)
        batch_data = [df.iloc[i] for i in batch_indices]
        
        X1 = []
        X2 = []
        Y = []

        for data in batch_data:
            left_img = preprocess_image(data['left'], is_left=True) 

            for i in range(20):
                X1.append(left_img)
                X2.append(preprocess_image(data[f'c{i}'], is_left=False))
                
                # Assign a 1 if this is the ground truth, otherwise 0
                if data[f'c{i}'] == data['groundtruth']:
                    Y.append([1 if j == i else 0 for j in range(20)])
                else:
                    Y.append([0 for _ in range(20)])

        X1 = np.vstack(X1)
        X2 = np.vstack(X2)
        Y = np.array(Y)

        yield [X1, X2], Y

In [ ]:
model = build_model(base_model)

epochs = 10
batch_size = 32
steps_per_epoch = len(train_df) // batch_size

model.fit(data_generator(train_df, batch_size), 
          epochs=epochs, 
          steps_per_epoch=steps_per_epoch,
          validation_data=data_generator(val_df, batch_size),
          validation_steps=len(val_df) // batch_size)

Epoch 1/10


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dot
from tensorflow.keras.preprocessing import image

base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
base_model.trainable = False

def preprocess_image(img_name, folder):
    img_path = f"./train/{folder}/{img_name}.jpg"
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(expanded_img_array)

def extract_embedding(img_name, folder):
    img = preprocess_image(img_name, folder)
    return base_model.predict(img)

df = pd.read_csv('newTrain.csv')

left_embeddings = [extract_embedding(row['left'], 'left') for _, row in df.iterrows()]
right_embeddings = [[extract_embedding(row[f'c{i}'], 'right') for i in range(20)] for _, row in df.iterrows()]

left_embeddings = np.array(left_embeddings)
right_embeddings = np.array(right_embeddings)

labels = np.zeros((len(df), 20))
for idx, row in df.iterrows():
    for i in range(20):
        if row[f'c{i}'] == row['groundtruth']:
            labels[idx, i] = 1

left_input = Input(shape=(2048,))
right_input = Input(shape=(20, 2048))
similarity = Dense(20, activation='softmax')(left_input)
dot_product = Dot(axes=1)([similarity, right_input])
model = Model(inputs=[left_input, right_input], outputs=dot_product)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit([left_embeddings, right_embeddings], labels, epochs=20, batch_size=32, validation_split=0.2)



1/1 [==============================] - 0s 97ms/step


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dot, Dropout
from tensorflow.keras.preprocessing import image

base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
base_model.trainable = False

def preprocess_image(img_name, folder):
    img_path = f"./train/{folder}/{img_name}.jpg"
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(expanded_img_array)

def extract_embedding(img_name, folder):
    img = preprocess_image(img_name, folder)
    return base_model.predict(img)

df = pd.read_csv('newTrain.csv')

left_embeddings = [extract_embedding(row['left'], 'left') for _, row in df.iterrows()]
right_embeddings = [[extract_embedding(row[f'c{i}'], 'right') for i in range(20)] for _, row in df.iterrows()]

left_embeddings = np.array(left_embeddings)
right_embeddings = np.array(right_embeddings)

left_embeddings = np.squeeze(left_embeddings, axis=1)
right_embeddings = np.reshape(right_embeddings, (-1, 20, 2048))

labels = np.zeros((len(df), 20))
for idx, row in df.iterrows():
    for i in range(20):
        if row[f'c{i}'] == row['groundtruth']:
            labels[idx, i] = 1

left_input = Input(shape=(2048,))
right_input = Input(shape=(20, 2048))

x = Dense(1024, activation='relu')(left_input)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
similarity = Dense(20, activation='softmax')(x)

similarity_expanded = tf.keras.layers.Reshape((20, 1))(similarity) # Change its shape to (20, 1)
scores = tf.keras.layers.Multiply()([right_input, similarity_expanded])
scores = tf.keras.layers.Lambda(lambda x: tf.reduce_sum(x, axis=2))(scores)

model = Model(inputs=[left_input, right_input], outputs=scores)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit([left_embeddings, right_embeddings], labels, epochs=20, batch_size=32, validation_split=0.3)


1/1 [==============================] - 0s 249ms/step
Epoch 1/20
5/5 [==============================] - 1s 58ms/step - loss: 4.2660 - accuracy: 0.0504 - val_loss: 3.5791 - val_accuracy: 0.0333
Epoch 2/20
5/5 [==============================] - 0s 33ms/step - loss: 3.5162 - accuracy: 0.1079 - val_loss: 3.3280 - val_accuracy: 0.0667
Epoch 3/20
5/5 [==============================] - 0s 35ms/step - loss: 3.0779 - accuracy: 0.1583 - val_loss: 3.2098 - val_accuracy: 0.0167
Epoch 4/20
5/5 [==============================] - 0s 35ms/step - loss: 2.6585 - accuracy: 0.2158 - val_loss: 3.2959 - val_accuracy: 0.0500
Epoch 5/20
5/5 [==============================] - 0s 35ms/step - loss: 2.2350 - accuracy: 0.3525 - val_loss: 3.2591 - val_accuracy: 0.0333
Epoch 6/20
5/5 [==============================] - 0s 36ms/step - loss: 2.2196 - accuracy: 0.3453 - val_loss: 3.3384 - val_accuracy: 0.0167
Epoch 7/20
5/5 [==============================] - 0s 39ms/step - loss: 1.8211 - accuracy: 0.4604 - val_loss: 3.41